In [ ]:
!pip install cohere pinecone-client PyPDF2 gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 707.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.2/249.2 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import cohere
import pinecone
import PyPDF2
import gradio as gr
import os
from pinecone import Pinecone, ServerlessSpec

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [ ]:
# Initialize Cohere
cohere_client = cohere.Client('SeiMH89JES2SBOmT811HSH9JC7kq9rc07pyTCqvT')

# Initialize Pinecone client
pc = Pinecone(api_key='db545199-08c0-4f4a-a285-79401be8375f')

# Define index name and dimension
index_name = 'qa-bot'

# Check if the index exists, create it if not
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=4096,
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'  # Set the appropriate region
        )
    )

# Connect to the index
index = pc.Index(index_name)

In [ ]:
# Function to read and extract text from PDF
def upload_pdf(file):
    pdf_reader = PyPDF2.PdfReader(file)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text() + "\n"
    return text

In [ ]:
# Function to ingest PDF text into Pinecone
def ingest_pdf(file):
    pdf_text = upload_pdf(file)

    # Split text into chunks if needed
    documents = pdf_text.split('\n\n')  # Adjust as needed for chunking

    # Create embeddings for each chunk
    embeddings = cohere_client.embed(texts=documents).embeddings

    # Insert embeddings into Pinecone
    for i, embed in enumerate(embeddings):
        index.upsert([(str(i), embed, {"text": documents[i]})])

    return "PDF content has been successfully ingested into Pinecone."

In [ ]:
# Retrieve relevant documents based on a query
def retrieve_relevant_docs(query):
    query_vector = cohere_client.embed(texts=[query]).embeddings[0]
    response = index.query(
        vector=query_vector,
        top_k=5,
        include_values=True,
        include_metadata=True
    )
    retrieved_docs = [match['metadata'] for match in response['matches']]
    return retrieved_docs

In [ ]:
# Generate answer based on the retrieved documents
def generate_answer(query, retrieved_docs):
    if retrieved_docs:
        context = ' '.join([doc['text'] for doc in retrieved_docs])
        prompt = f"Context: {context}\n\nQuestion: {query}\n\nPlease provide a direct answer based on the context above:"
    else:
        prompt = f"Question: {query}\n\nPlease provide a direct answer based on your knowledge."

    response = cohere_client.generate(
        model='command-r-plus',
        prompt=prompt,
        max_tokens=100,
        temperature=0.5
    )
    return response.generations[0].text.strip()

In [ ]:
# Combine document retrieval and answer generation
def qa_bot(query, pdf_file=None):
    if pdf_file is not None:
        # If a PDF is uploaded, ingest its content and retrieve relevant documents
        pdf_status = ingest_pdf(pdf_file)
        retrieved_docs = retrieve_relevant_docs(query)
        answer = generate_answer(query, retrieved_docs)
        return pdf_status, answer
    else:
        # If no PDF is uploaded, answer the question using general knowledge
        answer = generate_answer(query, [])
        return None, answer

In [ ]:
import gradio as gr

# Function to handle chatbot interaction
def chatbot_interface(query, pdf_file=None, chat_history=[]):
    pdf_status = None
    # If a PDF is uploaded, ingest its content and retrieve relevant documents
    if pdf_file is not None:
        pdf_status, answer = qa_bot(query, pdf_file)
    else:
        # If no PDF is uploaded, just answer the question using general knowledge
        pdf_status, answer = qa_bot(query)

    # Append the new question-answer pair to the chat history
    chat_history.append((query, answer))

    # Return the chat history along with PDF ingestion status (if applicable)
    return chat_history, "", pdf_status

# Gradio interface setup with custom CSS
with gr.Blocks(css=".small-upload-btn .btn {padding: 2px 8px;}") as ui:
    # Set up the header
    gr.Markdown("<h1 align='center'>QA Chatbot</h1>")

    # Display the chat history
    chatbot_output = gr.Chatbot(label="Chat History")

    # Input components: text box for questions, small file uploader for PDF, and submit button
    with gr.Row():
        # Question input box
        question_input = gr.Textbox(
            placeholder="Ask me any question",
            label="Your Question",
            show_label=False,
        )

        # Small PDF upload button
        pdf_uploader = gr.File(label="", elem_classes="small-upload-btn", type="filepath", show_label=False)

    # Submit button below the question input
    submit_btn = gr.Button(value="Submit", variant="primary")

    # Hidden textbox for managing chat history
    chat_history_state = gr.State([])

    # Status box for PDF ingestion
    pdf_status_box = gr.Textbox(label="PDF Status", placeholder="Upload a PDF to ingest it.", interactive=False)

    # Set the button functionality
    submit_btn.click(
        fn=chatbot_interface,
        inputs=[question_input, pdf_uploader, chat_history_state],
        outputs=[chatbot_output, question_input, pdf_status_box]
    )

# Launch the Gradio interface
ui.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d98afaf7485fc203b8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
